In [1]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [34]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re


def delevel_query(query_list):

    sub_query = []
    pos_delete = [len(query_list)-1]
    pos_join = []
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete.append(i)
        if line.startswith('FROM') and len(line.split(' ')) > 1:
            pos_join.append(i)
        elif line.startswith('FROM') and len(line.split(' ')) == 1:
            pos_join.append(i+1)
        if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
            pos_join.append(i+1)

    pos_join.append(min(pos_delete))

    sub_query = {}
    pos_join_list = iter(pos_join)
    next(pos_join_list)

    for i in range(len(pos_join)-1):
        if i < len(pos_join)-2:
            sub_query['sub_query_{}'.format(i)] = ' '.join(query_list[pos_join[i] : next(pos_join_list)-1])
        else:
            sub_query['sub_query_{}'.format(i)] = ' '.join(query_list[pos_join[i] : pos_join[-1]])
            
    return sub_query



def parse_sub_query(sub_query_list):
    sub_query = "\n".join(sub_query_list)
    sub_query = sub_query.lstrip('\n').lstrip(' ')
    
    formatter = column_parser.Parser(sub_query)
    formatted = formatter.format_query(sub_query)
    sub_query_list = formatted.split('\n')

    return sub_query_list


def has_child(sub_query_list):
    query_list = parse_sub_query(sub_query_list)
    query_list = delevel_query(query_list)
    return query_list, query_list != {}


In [35]:
query = """SELECT sfdc_accounts.platform, sfdc_accounts.mobile_os, sfdc_accounts.service_metadata,
sfdc_cases.account, sfdc_cases.num_requests, sfdc_cases.owner
FROM sfdc.accounts sfdc_accounts
LEFT JOIN (SELECT MAX(dt) FROM (SELECT dt FROM sfdc.oppty ) sfdc_oppty LEFT JOIN (SELECT dt FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE sfdc_cases.dt > '2020-04-03' AND sfdc_cases.dt < '2020-05-04' ORDER BY 1 GROUP BY 3 LIMIT 20
"""


## delevel_query

## level 0 -- level 1 -- level 2

In [82]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
# cte_queries = formatter.parse_cte(formatted_query)

In [84]:
# query_list = formatted_query.split('\n')
# has_subquery = True
# counter = 0
# while has_subquery:
#     counter += 1
#     sub_query_list, has_subquery = has_child(query_list)
#     query_list = sub_query_list
#     print(has_subquery, counter, '\n'.join(query_list),  '\n')

In [85]:
query_list_0 = formatted_query.split('\n')

In [86]:
query_list_0

['SELECT sfdc_accounts.platform,',
 '       sfdc_accounts.mobile_os,',
 '       sfdc_accounts.service_metadata,',
 '       sfdc_cases.account,',
 '       sfdc_cases.num_requests,',
 '       sfdc_cases.owner',
 'FROM sfdc.accounts sfdc_accounts',
 'LEFT JOIN',
 '  (SELECT MAX(dt)',
 '   FROM',
 '     (SELECT dt',
 '      FROM sfdc.oppty) sfdc_oppty',
 '   LEFT JOIN',
 '     (SELECT dt',
 '      FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt',
 "WHERE sfdc_cases.dt > '2020-04-03'",
 "  AND sfdc_cases.dt < '2020-05-04'",
 'ORDER BY 1',
 'GROUP BY 3',
 'LIMIT 20']

In [125]:
sub_query_1 = delevel_query(query_list_0)
sub_query_1

{'sub_query_0': 'FROM sfdc.accounts sfdc_accounts',
 'sub_query_1': "  (SELECT MAX(dt)    FROM      (SELECT dt       FROM sfdc.oppty) sfdc_oppty    LEFT JOIN      (SELECT dt       FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt WHERE sfdc_cases.dt > '2020-04-03'   AND sfdc_cases.dt < '2020-05-04'"}

In [126]:
sub_query_1 = sub_query_1['sub_query_1'].lstrip('\n \(')
sub_query_1

"SELECT MAX(dt)    FROM      (SELECT dt       FROM sfdc.oppty) sfdc_oppty    LEFT JOIN      (SELECT dt       FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt WHERE sfdc_cases.dt > '2020-04-03'   AND sfdc_cases.dt < '2020-05-04'"

In [129]:
formatter = column_parser.Parser(sub_query_1)
formatted_sub_query_1 = formatter.format_query(sub_query_1)
print(formatted_sub_query_1)

SELECT MAX(dt)
FROM
  (SELECT dt
   FROM sfdc.oppty) sfdc_oppty
LEFT JOIN
  (SELECT dt
   FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE sfdc_cases.dt > '2020-04-03'
  AND sfdc_cases.dt < '2020-05-04'


In [130]:
delevel_query(formatted_sub_query_1.split('\n'))

{'sub_query_0': '  (SELECT dt    FROM sfdc.oppty) sfdc_oppty',
 'sub_query_1': "  (SELECT dt    FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt WHERE sfdc_cases.dt > '2020-04-03'"}